In [1]:
import numpy as np # linear alra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

from sklearn.utils import shuffle


from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.metrics import *


import tensorflow as tf
from flask import Flask,render_template,session,url_for,redirect,request,jsonify
from flask_wtf import FlaskForm
import os
import os 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

from tensorflow.python.keras.models import load_model
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
app = Flask(__name__)


In [3]:
target_names = ['COVID-19', 'NORMAL', 'Viral Pneumonia']

models=[]
models.append(load_model('covid-resnet50.h5'))
models.append(load_model('covid-mobilenet.h5'))
models.append(load_model('covid-mobilenetV2.h5'))
models.append(load_model('covid-DenseNet.h5'))

model_index = {
  "resNet50": 0,
  "mobileNet": 1,
  "mobileNetV2": 2,
  "denseNet" : 3
}

In [4]:
def predict_image(model_name,img):
    img=cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_AREA)
    img=img/255.0
    img=img.reshape(1,64,64,3)
    my_model=models[model_index[model_name]]
    x=my_model.predict(img)
    y=x.argmax(axis=1)
    x=x[0][y][0]
    x=x*100
    x=round(x,2)
    answer = "The " + model_name+" model predicts the given image as " + target_names[y[0]] + " with a confidence level:"+ str(x)+"%"
    return answer

In [5]:
@app.route('/predict', methods=['POST'])
def prediction():
    if request.method == 'POST':
        imagefile =np.array(Image.open(request.files['image']))
        
        resNet50 = predict_image("resNet50",imagefile)
        mobileNet = predict_image("mobileNet",imagefile)
        mobileNetV2 = predict_image("mobileNetV2",imagefile)
        denseNet = predict_image("denseNet",imagefile)
        return {
            "ResNet50": resNet50,
            "MobileNet": mobileNet,
            "MobileNetV2": mobileNetV2,
            "DenseNet": denseNet
        } 



In [ ]:
 
if __name__ == '__main__':
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Aug/2020 20:52:44] "POST /predict HTTP/1.1" 200 -
